In [ ]:
using Plots
# using Revise
using LatticeQM

In [ ]:
?LatticeQM

# Tutorial: Lattices

A lattice is characterized by a set of lattice vectors $\vec{a}_1$, $\dots$, $\vec{a}_d$ in real space $\mathbb{R}^{D>d}$. If $d<D$, we can always find additional basis vectors to span the space that is not covered by the lattice, i.e. $\vec{e}_{d+1}$, $\dots$, $\vec{e}_D$.

These lattice vectors define a *primitive unit cell* that contains a finite number $N$ of atoms/orbitals at positions $\vec{r}_1$, $\dots$, $\vec{r}_N$. These positions can be written in *fractional coordinates* as $\vec{r}_i = \sum_{j=1}^d x_j \vec{a}_j + \sum_{j=d+1}^D x_j \vec{e}_j$.

In [ ]:
import LatticeQM.Structure.Lattices: Lattice, addbasis!, addorbital!, addorbitals!, addextra!

In [ ]:
## Define real-space basis
lat = Lattice() # 0D lattice
addbasis!(lat, [1,0,0]) # 1D lattice
addbasis!(lat, [0,1,0]) # 2D lattice
addbasis!(lat, [0,0,1], :finite) # 2D lattice with z-coordinates
addextra!(lat, "sublattice") # non-spatial coordinate

## Adding atoms/orbitals to the lattice
addorbital!(lat, [0,   0,  -1, 0]) # must be vector of correct length (here 2 + 1 + 1)
addorbital!(lat, [1/2, 1/2, 1, 1])

In [ ]:
## Short version
lat = Lattice([[1,0,0] [0,1,0] [0, 0, 1]]; periodic=2, extra=["sublattice"])
addorbitals!(lat, [[0,   0,  -1, 0] [1/2, 1/2, 1, 1]]) # must be matrix with coordinates as columns

## Shorter version
lat = Lattice(2,3; extra=["sublattice"]) # 2D lattice in 3D space with 1 extra coordinate "sublattice"
addorbitals!(lat, [[0,   0,  -1, 0] [1/2, 1/2, 1, 1]]) # must be matrix with coordinates as columns


In [ ]:
kpoints = Dict(
    "Γ" => [ [0.0; 0.0], "\$\\Gamma\$" ],
    "M1" => [ [1/2;  0.0], "M"],
    "M2" => [ [0.0;  1/2], "M'"],
    "Z" =>  [ [1/2;  1/2], "Z"]
)

# Pre-defined lattice objects

In [ ]:
lat = Geometries.square()
display(lat)
plot(lat; supercell=[3,3])

In [ ]:
lat = Geometries.honeycomb()
slat = Structure.Lattices.superlattice(lat, [[5, -5] [5, 5]])
display(lat)
plot(slat, "sublattice")

# Superlattice

In [ ]:
lat = Geometries.honeycomb()
slat = Structure.Lattices.superlattice(lat, [[1, 0] [0, 6]])

display(slat)
plot(slat, "sublattice"; supercell=[12,0])

In [ ]:
lat = Geometries.honeycomb()
slat = Structure.Lattices.superlattice(lat, [[1, 1] [6, -6]])

display(slat)
plot(slat, "sublattice"; supercell=[12,0])

In [ ]:
import LinearAlgebra: norm

function honeycombholes(; N=4,rad=0.2, rhombic=false)
    lat = Geometries.honeycomb()
    Lattices.translate!(lat, [1/3,1/3,0])
    lat = Lattices.superlattice(lat, [[N,0] [0,N]])
    Lattices.foldPC!(lat)#; shift=[1/3,-1/3,0])
    filterind = map(x->norm(x)>rad*norm(Lattices.getA(lat)[:,1]), eachcol(Lattices.positions(lat))) |> findall
    lat.spacecoordinates = lat.spacecoordinates[:,filterind]
    lat.extracoordinates = lat.extracoordinates[:,filterind]

    if rhombic
        Lattices.translate!(lat, [-1/2,-1/2,0])
        lat.spacecoordinates = mod.(lat.spacecoordinates, 1.0)
    end

    lat
end

lat = honeycombholes(N=12,rad=0.27,rhombic=false)

println("No of atoms: ", Lattices.countorbitals(lat))

plot(lat, "sublattice"; markersize=2, supercell=[2,2])
# filterind = Lattices.positions()

# Reducing lattice dimension

In [ ]:
lat = Geometries.honeycomb()
lat_armchair = Structure.Lattices.reduceto1D(lat, [[1, 1] [6, -6]])

display(lat_armchair)
plot(lat_armchair, "sublattice"; supercell=[20])

# Twisted lattices

In [ ]:
lat = Geometries.honeycomb_twisted(10)

In [ ]:
plot(lat, 3; supercell=1, markersize=2.5)